In [10]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
import random

DATA_PATH = 'D:\\Data Mining\\data\\'

testdata = pd.read_csv(DATA_PATH + "bigmerge.csv")

In [11]:
testdata = testdata.drop('gender', axis=1)
testdata = testdata.drop('age', axis=1)
testdata = testdata.drop('Unnamed: 0', axis=1)

In [12]:

#testdata needs to be without gender and age-group
dataframe = testdata

In [13]:
#23290 IDs
#1466 lenght of one Dataset
#23290 * 1466 = 3506672

#traindata length encodearray 22766980

#Tainset = 15530
#Testset = 7760

#Groups in Labellists
################
#0 = M22-
#1 = M23-26
#2 = M27-28
#3 = M29-31
#4 = M32-38
#5 = M39+
#6 = F23-
#7 = F24-26
#8 = F27-28
#9 = F29-32
#10 = F33-42
#11 = F43+

print set(dataframe['group'])

set(['M39+', 'F27-28', 'M32-38', 'M23-26', 'F43+', 'M22-', 'F33-42', 'F23-', 'F24-26', 'F29-32', 'M27-28', 'M29-31'])


In [14]:
#returns a list of Label Names label 0 = 'M22-' 1 = ''
def getLabelNames():
    labelNames = ['M22-','M23-26','M27-28','M29-31','M32-38','M39+','F23-','F24-26','F27-28','F29-32','F33-42','F43+']
    
    return labelNames

In [15]:
def getTrainDataframe(dataframe):
    idList = dataframe[dataframe.columns[0]].unique()
    trainset = pd.DataFrame(index=range(0), columns=['device_id','group','phone_brand','device_model','category'])
    
    for i in range(15530):
        tmpdf = dataframe.loc[dataframe[dataframe.columns[0]] == idList[i]]
        frames = [tmpdf, trainset]
        trainset = pd.concat(frames)
    
    return trainset

In [16]:
def getTestDataframe(dataframe):
    idList = dataframe[dataframe.columns[0]].unique()
    testset = pd.DataFrame(index=range(0), columns=['device_id','group','phone_brand','device_model','category'])
    
    for i in range(15530,23289):
        tmpdf = dataframe.loc[dataframe[dataframe.columns[0]] == idList[i]]
        frames = [tmpdf, testset]
        testset = pd.concat(frames)
    
    return testset

In [17]:
def getLabelList(dataframe):
    #get all IDs, make a list
    idList = dataframe[dataframe.columns[0]].unique()
    #enumerate idList and create a list with age group
    labelList = []
    for i in range(len(idList)):
        value = dataframe.loc[dataframe['device_id'] == idList[i], 'group'].values[0]
        labelList.append(value)
        
    #replace all values in labelList with GroupValues int between 0-11   
    for v in range(len(labelList)):
        if labelList[v] == 'M22-':
            labelList[v]=0
        elif labelList[v] == 'M23-26':
            labelList[v]=1
        elif labelList[v] == 'M27-28':
            labelList[v]=2    
        elif labelList[v] == 'M29-31':
            labelList[v]=3   
        elif labelList[v] == 'M32-38':
            labelList[v]=4
        elif labelList[v] == 'M39+':
            labelList[v]=5
        elif labelList[v] == 'F23-':
            labelList[v]=6
        elif labelList[v] == 'F24-26':
            labelList[v]=7
        elif labelList[v] == 'F27-28':
            labelList[v]=8
        elif labelList[v] == 'F29-32':
            labelList[v]=9
        elif labelList[v] == 'F33-42':
            labelList[v]=10
        elif labelList[v] == 'F43+':
            labelList[v]=11
        
    return labelList

In [18]:
#create traindataframe
traindataframe = getTrainDataframe(dataframe)

#create trainlabellist
trainlabellist = getLabelList(traindataframe)

#create testdataframe
testdataframe = getTestDataframe(dataframe)

#create testlabellist
testlabellist = getLabelList(testdataframe)

In [19]:
#takes dataframe and returns a random dataset by id
def getRandomDataset(dataframe):
    #get all IDs, make a list
    idList = dataframe[dataframe.columns[0]].unique()
    #get a random id from list
    randomID = idList[random.randint(0,len(idList)-1)]
    #get the random dataset by id
    randomDataSet = dataframe.loc[dataframe[dataframe.columns[0]] == randomID]
   
    return randomDataSet

In [20]:
dataframe = testdata

#takes dataframe and returns python dictionary with unique values and headers
def createMatrixSizeDic(dataframe):
    #slice our id away
    dataframe = dataframe.drop(dataframe.columns[0], axis=1)
    dataframe = dataframe.drop('group', axis=1)
    #get headers to list
    headers = list(dataframe.columns.values)
    #count values distinct
    sizeDic = {headers[x]:dataframe[headers[x]].unique() for x in range(len(headers))}
    
    return sizeDic
    

In [21]:
#creates a one hot encode array
def createMainDataOneHotEncode(dataframeall, dataframe):
    oneHotEncodeArray = []
    
    #get all IDs, make a list
    idList = dataframe[dataframe.columns[0]].unique()
    #get the dataframe size Dictionary
    sizeDic = createMatrixSizeDic(dataframeall)

    for ind in range(len(idList)):
        
        #get the random dataset by id
        DataSet = dataframe.loc[dataframe[dataframe.columns[0]] == idList[ind]]
        
        #get random Group
        #DataSetGroup = DataSet['group'].unique()
        #DataSetGroupStr = randomDataSetGroup[0]
    
#    print randomDataSetGroupStr

        #print DataSet


        #drop the id row of the random Dataset
        DataSet = DataSet.drop(DataSet.columns[0], axis=1)
    
        #drop group Row
        DataSet = DataSet.drop('group', axis=1)
    
        #get headers of randomDataSet
        headers = list(DataSet.columns.values)
    
        #create utilityLists oneHotEncodeDataset with sizeDic Parameters and randomDataset
        tmpList = []
        for key, value in sizeDic.iteritems():
            for x in value:
                tmpList.append(x)
            
#   print tmpList
    
        tmpList2 = []
        for column, row in DataSet.iterrows():
            for i in range(len(headers)):
                tmpList2.append(row[headers[i]])
            
        #remove duplicates
        tmpList2 = set(tmpList2)
        
#   print tmpList2
    
        #replace in tmpList all values with 1 when found in tmpList2 the rest values with 0
        for index, item in enumerate(tmpList2):
            for ind, val in enumerate(tmpList):
                if (item == val):
                    tmpList[ind] = 1
    
        for index, item in enumerate(tmpList):
            if (item != 1):
                tmpList[index] = 0
            
        oneHotEncodeArray.extend(tmpList)
#   print len(oneHotEncodeArray[randomDataSetGroupStr:tmpList])
    
    return oneHotEncodeArray

In [22]:
#create Traindata and Testdata as numpy array

x = createMainDataOneHotEncode(dataframe, traindataframe)

TrainData = np.array(x)

print len(x)

y = createMainDataOneHotEncode(dataframe, testdataframe)

TestData = np.array(y)

print len(y)

22766980
11374694


In [21]:
#save the trainfile and testfile as .npy array  
    
np.save(DATA_PATH + 'trainData.npy', TrainData, allow_pickle=True, fix_imports=True)
np.save(DATA_PATH + 'trainLabels.npy', trainlabellist, allow_pickle=True, fix_imports=True)

np.save(DATA_PATH + 'testData.npy', TestData, allow_pickle=True, fix_imports=True)
np.save(DATA_PATH + 'testLabels.npy', testlabellist, allow_pickle=True, fix_imports=True)


In [23]:
BigTestData = np.array()

TypeError: Required argument 'object' (pos 1) not found

In [24]:
#save the labelnames
labelNames = getLabelNames()

np.save('/home/daniel/labelNames.npy', labelNames, allow_pickle=True, fix_imports=True)

IOError: [Errno 2] No such file or directory: '/home/daniel/labelNames.npy'

In [25]:
#creates a one hot encode array
def createRandomOneHotEncode(dataframe):
    #get the dataframe size Dictionary
    sizeDic = createMatrixSizeDic(dataframe)
    #get a random Dataset
    randomDataSet = getRandomDataset(dataframe)
    #get random Group
    randomDataSetGroup = randomDataSet['group'].unique()
    randomDataSetGroupStr = randomDataSetGroup[0]
    
#    print randomDataSetGroupStr
    #drop the id row of the random Dataset
    randomDataSet = randomDataSet.drop(randomDataSet.columns[0], axis=1)
    #drop group Row
    randomDataSet = randomDataSet.drop('group', axis=1)
    #get headers of randomDataSet
    headers = list(randomDataSet.columns.values)
    #create utilityLists oneHotEncodeDataset with sizeDic Parameters and randomDataset
    tmpList = []
    for key, value in sizeDic.iteritems():
        for x in value:
            tmpList.append(x)
            
#   print tmpList22
    
    tmpList2 = []
    for column, row in randomDataSet.iterrows():
        for i in range(len(headers)):
            tmpList2.append(row[headers[i]])
            
    #remove duplicates
    tmpList2 = set(tmpList2)
#   print tmpList2
    
    #replace in tmpList all values with 1 when found in tmpList2 the rest values with 0
    for index, item in enumerate(tmpList2):
        for ind, val in enumerate(tmpList):
            if (item == val):
                tmpList[ind] = 1
    
    for index, item in enumerate(tmpList):
        if (item != 1):
            tmpList[index] = 0
            
    oneHotEncodeArray = {randomDataSetGroupStr:tmpList}
#   print len(oneHotEncodeArray[randomDataSetGroupStr:tmpList])
    
    return oneHotEncodeArray

In [26]:
testdataframe = getTestDataframe(dataframe)

In [27]:
print getLabelList(dataframe)


[5, 5, 5, 5, 10, 5, 9, 7, 11, 3, 4, 5, 8, 9, 5, 2, 4, 3, 4, 4, 5, 1, 1, 5, 5, 4, 5, 11, 3, 4, 3, 9, 11, 9, 4, 11, 10, 5, 4, 4, 3, 5, 5, 11, 11, 4, 4, 5, 11, 5, 4, 3, 11, 11, 1, 9, 4, 5, 5, 5, 2, 4, 5, 3, 4, 5, 5, 10, 3, 3, 2, 4, 4, 4, 4, 4, 0, 10, 7, 5, 4, 11, 4, 3, 4, 4, 5, 4, 11, 3, 4, 3, 3, 5, 3, 10, 11, 0, 5, 3, 10, 1, 3, 4, 4, 4, 5, 3, 1, 3, 10, 7, 3, 3, 3, 4, 11, 4, 5, 11, 2, 2, 5, 7, 5, 5, 5, 11, 3, 1, 5, 9, 5, 9, 10, 4, 4, 4, 5, 3, 9, 4, 4, 3, 5, 4, 10, 4, 5, 5, 11, 1, 5, 5, 9, 8, 11, 5, 2, 3, 2, 5, 5, 5, 10, 4, 3, 3, 11, 1, 11, 4, 4, 4, 2, 5, 3, 3, 4, 5, 11, 5, 4, 9, 11, 4, 8, 7, 5, 10, 1, 3, 10, 11, 7, 3, 4, 4, 5, 5, 4, 5, 1, 4, 3, 5, 10, 3, 5, 5, 1, 5, 4, 4, 3, 4, 10, 3, 7, 9, 5, 11, 4, 5, 3, 5, 5, 2, 5, 5, 11, 5, 5, 5, 5, 5, 3, 4, 11, 3, 1, 5, 1, 5, 5, 4, 5, 0, 4, 3, 4, 4, 3, 2, 2, 3, 9, 5, 11, 5, 4, 5, 4, 2, 1, 5, 0, 5, 4, 5, 5, 5, 10, 2, 11, 5, 5, 3, 5, 4, 10, 5, 3, 3, 5, 5, 5, 5, 3, 5, 4, 3, 5, 5, 4, 11, 9, 5, 5, 11, 4, 5, 5, 5, 5, 1, 4, 5, 5, 0, 5, 2, 1, 5, 5, 4, 4, 2, 

In [28]:
print len(getIDList(dataframe))

NameError: name 'getIDList' is not defined

In [ ]:
getRandomDataset(dataframe)

In [ ]:

OneHot = createOneHotEncode(dataframe)

print len(OneHot.values()[0])

#1466

print OneHot